In [1]:
import google.generativeai as genai
from db_utils import csv_read,query_database
import re

c:\mypython\project_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genai.configure(api_key="xxx")

In [3]:
def generate_sql_from_nl(user_input):
    csv_string = csv_read()
    system_prompt = f'''
        1. 你是一個財務資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文，結束請記得給分號。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現資料庫中沒有的英文名稱。
        5. 請務必生成完整SQL語法，並一定要在首欄加入年或季度的欄位，例如seasonid。
        6. **SQL 中報表的欄位名絕對不要使用別稱 (AS)。** 
        7. **請務必嚴格遵守以上所有指令。**
       '''

    user_prompt = f'''
        1. **資料庫結構、表名稱、欄位名稱請務必參考以下內容:{csv_string}**
        2. 一定不能生成沒有的sql表名與欄名，欄位名稱請盡量比對中文對照!
        3. **請務必要判斷是否需要跨表查詢，如需要請使用join指令。**
        4. 另外時間只有年與季度seasonsID，例如2010年第一季格式為201001、2010年第二季格式為201002、
        2020年第三季格式為202003、2020年第四季格式為202004
        5. **請務必嚴格遵守不要使用別稱 (AS)**
                   '''
    model=genai.GenerativeModel(model_name="gemini-1.5-pro")

    prompt_content = [
        {"role": "user", "parts": [system_prompt]},
        {"role": "user", "parts": [user_input + user_prompt]}
    ]
    response = model.generate_content(
        contents = prompt_content,
        generation_config = genai.types.GenerationConfig(
            max_output_tokens=150,
            temperature=0.1
        )
    )
    sql_response = response.text
    return sql_response


模型可用系列:GPT、Gemini

In [4]:
def extract_sql_queries(sql_response):
    matches = re.findall(r'SELECT.*?;', sql_response, re.IGNORECASE | re.DOTALL) 
    if matches:
        # 移除每個匹配項中的換行符號 
        matches = [match.replace('\n', ' ') for match in matches]
        return matches 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [5]:
def classify_user_input(user_input:str): 
    # 初次調用判斷是否需要function calling

    model=genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt_content = [
        {"role": "user", 
         "parts":'''1.請判斷使用者的輸入是否需要連線到「財務報表」資料庫查詢，若需要請回答finvision即可!
                    2.若使用者的輸入是關於分析財務指標，請返回analysis即可!
                    3.若都不屬於查詢與分析範圍，只要回答NO即可!
                    4.財務報表資料庫內容包含資產負債表、損益表、現金流量表，其餘公司資料都沒有。'''},
        {"role": "user", "parts": user_input}
    ]
    
    
    response = model.generate_content(contents = prompt_content)
    
    
    print('===== 初次調用LLM判斷 =====')
    if 'finvision' in response.text.lower():
        retries = 0
        sql_response = generate_sql_from_nl(user_input)
        while retries <= 5:
            sql_query = extract_sql_queries(sql_response)
            print(f'生成的SQL指令: {sql_query}')
            report = query_database(sql_query)
            if "查詢成功" in report:
                # 成功，回傳結果
                return f"finvision:{report}"
            else:
                # 失敗，再試一次
                print(f'SQL查詢失敗: {report}')
                sql_response = generate_sql_from_nl(user_input + report +'若有錯誤請務必重新確認欄位與表的關係')
                retries += 1
            print(f"第{retries}次重試")
        
        # 如果達到最大重試次數，則返回錯誤訊息
        return "finvision: SQL查詢多次失敗，請稍後再試。"

    else: 
        # 一般對話處理
        return(response.text)
         


In [6]:
def main():
    user_input ="請給我2023年各季度之利息收入、法定盈餘公積、銷貨收入"
    report = classify_user_input(user_input)
    print(report)

In [7]:
main()

===== 初次調用LLM判斷 =====
生成的SQL指令: ["SELECT     seasonsID,     II,     LR,     NSR FROM     pl WHERE     seasonsID LIKE '2023%';"]
連線成功
SQL查詢失敗: {"status": "查詢失敗", "message": "SQL query error: (1054, \"Unknown column 'LR' in 'field list'\")"}
第1次重試
生成的SQL指令: ["SELECT seasonsID, II, LR, NSR FROM pl JOIN seasons ON pl.seasonsID = seasons.seasonsID WHERE pl.seasonsID LIKE '2023%';"]
連線成功
SQL查詢失敗: {"status": "查詢失敗", "message": "SQL query error: (1052, \"Column 'seasonsID' in field list is ambiguous\")"}
第2次重試
生成的SQL指令: ["SELECT seasons.seasonsID, pl.II, bs.LR, pl.NSR FROM pl JOIN bs ON pl.seasonsID = bs.seasonsID JOIN seasons ON pl.seasonsID = seasons.seasonsID WHERE seasons.seasonsID LIKE '2023%';"]
連線成功
finvision:{"status": "查詢成功", "data": [[{"seasonsID": "202301", "II": 13617818, "LR": 311146899, "NSR": 0}, {"seasonsID": "202302", "II": 27289839, "LR": 311146899, "NSR": 0}, {"seasonsID": "202303", "II": 42885113, "LR": 311146899, "NSR": 0}, {"seasonsID": "202304", "II": 60293901, "LR": 311